In [3]:
!pip install imgaug opencv-python numpy pillow matplotlib


   ---------------------------------------- 0.0/948.0 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/948.0 kB ? eta -:--:--
   --------------------------------- ------ 786.4/948.0 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 948.0/948.0 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 3.4 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 3.7 MB/s eta 0:00:00


In [1]:
import os
import numpy as np
from PIL import Image
import imgaug.augmenters as iaa

# Load preprocessed fingerprint data
X = np.load("X_fingerprint.npy")  # Shape: (num_images, 128, 128, 1)
y = np.load("y_fingerprint.npy")  # Labels

# Define augmentation pipeline
augmenters = iaa.Sequential([
    iaa.Fliplr(0.5),         # 50% chance to flip horizontally
    iaa.Affine(rotate=(-15, 15)),  # Rotate between -15 to +15 degrees
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add noise
    iaa.Multiply((0.8, 1.2)),  # Random brightness
])

# Path to save augmented images
augmented_dir = r"D:\augmented_data"
os.makedirs(augmented_dir, exist_ok=True)

# Set augmentation factor
AUGMENT_FACTOR = 4  

aug_X, aug_y = [], []

print(f"Augmenting {len(X)} images...")

for i, img in enumerate(X):
    img = (img * 255).astype(np.uint8)  # Convert back to uint8 for augmentation
    
    for j in range(AUGMENT_FACTOR):
        aug_img = augmenters(image=img.squeeze())  # Apply augmentation
        aug_img = np.expand_dims(aug_img, axis=-1)  # Keep shape (128,128,1)
        
        aug_X.append(aug_img)
        aug_y.append(y[i])
        
        # Save the augmented image
        aug_img_pil = Image.fromarray(aug_img.squeeze())
        save_path = os.path.join(augmented_dir, f"img_{i}_{j}.png")
        aug_img_pil.save(save_path)

    if i % 500 == 0:
        print(f"Processed {i}/{len(X)} images...")

# Convert to NumPy arrays
aug_X = np.array(aug_X) / 255.0  # Normalize again
aug_y = np.array(aug_y)

# Save augmented dataset
np.save("X_fingerprint_aug.npy", aug_X)
np.save("y_fingerprint_aug.npy", aug_y)

print(f"✅ Augmented dataset saved! Total images: {len(aug_X)}")


Augmenting 6000 images...
Processed 0/6000 images...
Processed 500/6000 images...
Processed 1000/6000 images...
Processed 1500/6000 images...
Processed 2000/6000 images...
Processed 2500/6000 images...
Processed 3000/6000 images...
Processed 3500/6000 images...
Processed 4000/6000 images...
Processed 4500/6000 images...
Processed 5000/6000 images...
Processed 5500/6000 images...
✅ Augmented dataset saved! Total images: 24000
